In [106]:
import pandas as pd
import numpy as np
from kaggle.api.kaggle_api_extended import KaggleApi
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [108]:
api = KaggleApi()
api.authenticate

<bound method KaggleApi.authenticate of <kaggle.api.kaggle_api_extended.KaggleApi object at 0x0000028E6D76B680>>

In [110]:
dataset = "shivamb/real-or-fake-fake-jobposting-prediction"
api.dataset_download_files(dataset,path=".",unzip=True)

print("Dataset downloaded and extracted")

Dataset URL: https://www.kaggle.com/datasets/shivamb/real-or-fake-fake-jobposting-prediction
Dataset downloaded and extracted


In [112]:
df = pd.read_csv('fake_job_postings.csv')
df

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,17876,Account Director - Distribution,"CA, ON, Toronto",Sales,NaN,Vend is looking for some awesome new talent to...,Just in case this is the first time you’ve vis...,To ace this role you:Will eat comprehensive St...,What can you expect from us?We have an open cu...,0,1,1,Full-time,Mid-Senior level,NaN,Computer Software,Sales,0
17876,17877,Payroll Accountant,"US, PA, Philadelphia",Accounting,NaN,WebLinc is the e-commerce platform and service...,The Payroll Accountant will focus primarily on...,- B.A. or B.S. in Accounting- Desire to have f...,Health &amp; WellnessMedical planPrescription ...,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Internet,Accounting/Auditing,0
17877,17878,Project Cost Control Staff Engineer - Cost Con...,"US, TX, Houston",NaN,NaN,We Provide Full Time Permanent Positions for m...,Experienced Project Cost Control Staff Enginee...,At least 12 years professional experience.Abil...,NaN,0,0,0,Full-time,NaN,NaN,NaN,NaN,0
17878,17879,Graphic Designer,"NG, LA, Lagos",NaN,NaN,NaN,Nemsia Studios is looking for an experienced v...,1. Must be fluent in the latest versions of Co...,Competitive salary (compensation will be based...,0,0,1,Contract,Not Applicable,Professional,Graphic Design,Design,0


In [114]:
print(df['fraudulent'].value_counts())

fraudulent
0    17014
1      866
Name: count, dtype: int64


In [116]:
# equalizing the data
num_zeros_to_keep = 866
current_zeros = df[df['fraudulent'] == 0]
eligible_for_deletion = current_zeros[current_zeros['fraudulent'] == 0]
num_to_delete = len(current_zeros) - num_zeros_to_keep
rows_to_delete = eligible_for_deletion.sample(n=num_to_delete, random_state=42)
df.drop(index=rows_to_delete.index, inplace=True)

In [118]:
print(df['fraudulent'].value_counts())

fraudulent
0    866
1    866
Name: count, dtype: int64


In [120]:
df.shape

(1732, 18)

In [122]:
df.isnull().sum()

job_id                    0
title                     0
location                 33
department             1081
salary_range           1366
company_profile         704
description               1
requirements            286
benefits                686
telecommuting             0
has_company_logo          0
has_questions             0
employment_type         418
required_experience     774
required_education      847
industry                507
function                661
fraudulent                0
dtype: int64

In [124]:
df = df[['title', 'description', 'requirements', 'company_profile', 'benefits', 'fraudulent']]

In [126]:
df = df.copy()
df.fillna('', inplace=True)

In [128]:
df['text'] = df['title']+' '+df['description']+' '+df['requirements']+' '+df['company_profile']+' '+df['benefits']
df['text']

5        Accounting Clerk Job OverviewApex is an enviro...
9        Customer Service Associate - Part Time  The Cu...
55       Junior HR Marketing Manager We are Netguru and...
77       Mid-Senior .NET or Xamarin Developer DataFinch...
98       IC&E Technician IC&amp;E Technician | Bakersfi...
                               ...                        
17827    Student Positions Part-Time and Full-Time. Stu...
17828    Sales Associate LEARN TO EARN AN EXECUTIVE LEV...
17829    Android Developer inFullMobile Sp. z o.o. is a...
17830    Payroll Clerk  JOB DESCRIPTIONWe are seeking a...
17831    Furniture mover earn $500 to $1000 a week as a...
Name: text, Length: 1732, dtype: object

In [130]:
X = df['text']
y = df['fraudulent']

In [132]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [134]:
tfidf = TfidfVectorizer(stop_words='english', max_df=0.7)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [136]:
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [138]:
y_pred = model.predict(X_test)

In [140]:
acc = accuracy_score(y_test,y_pred)
print("\n Accuracy = ",acc)


 Accuracy =  0.9250720461095101


In [142]:
# Packing model and vectorizer
import joblib
joblib.dump(model, 'jobs.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')

['vectorizer.pkl']